In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#from model_prediction import Replicator
from pneumococcus_data import data_observation
from Jacobian import jacobian_func

In [2]:
# import Particle Swarm Optimization algorithm
from PSO_algo import Particle
from PSO_algo import PSO

### Description
The number of strains in Nepal is $n= 69$; and $\mu = 3.95$. We need to estimate the matrix $69 \times 69$ $\left(\alpha_{ij}\right)$ of Nepal. \
Since $\mu$ of Nepal is neither too large and nor too small, we estimate parameters such that the data observation (Nepal relative serotype prevalences) are: \
i) the equilibrium of the replicator dynamics, \
ii) a snapshot of the dynamics. \
In this program, we provide the process for case 
#### i) the equilibrium of the replicator dynamics.
The process is similar to the parameter estimation for Iran's $\alpha_{ij}$'s: \
i) From $\alpha_{ij}$'s estimated in Brazil, Denmark, Mozambique, and Iran, in Nepal's $\alpha_{ij}$ matrix, we insert $1901$ values estimated before. \
ii) SS: Calculate $\Lambda$ matrix and use PSO algorithm in range $\left[-10, 10\right]$ to estimate $2860$ remaining $\alpha_{ij}$'s such that data observation $z^*$ satisfies $\left\|\Lambda z^* - {z^*}^T\Lambda z^*\right\|_2$ attains its minimum and the equilibrium is stable, i.e. the Jacobian matrix corresponding to the equilibrium $\Lambda z^*$ has all strictly negative real-part eigenvalues. \
In PSO algorithm, we choose the range $\left[-10,10\right]$ for these $2860$ $\alpha_{ij}$'s. 

In [3]:
# data observation from 5 countries
Iran, Denmark, Brazil, Nepal, Mozambique = data_observation()

# import name of serotypes and their frequencies from Nepal
strain_number = len(Nepal.strain) # = 69
Nepal_strain = np.array(Nepal.strain)
Nepal_cases = np.array(Nepal.cases)

#ratio of single to co-colonization prevalence: mu value of Nepal
mu = 3.95

#import Brazil alpha & data
Brazil_alpha = np.loadtxt(open("Brazil_alpha.txt","rb"), delimiter = ",", skiprows = 0)
Brazil_strain = np.array(Brazil.strain)
#number_Brazil_strain = len(np.array(Brazil.strain))

#import Denmark alpha & data
Denmark_alpha = np.loadtxt(open("Denmark_alpha.txt","rb"), delimiter = ",", skiprows = 0)
Denmark_strain = np.array(Denmark.strain)

#import Mozambique alpha & data
Mozambique_alpha = np.loadtxt(open("Mozambique_alpha.txt","rb"), delimiter = ",", skiprows = 0)
Mozambique_strain = np.array(Mozambique.strain)

#import Iran alpha & data
Iran_alpha = np.loadtxt(open("Iran_alpha.txt","rb"), delimiter = ",", skiprows = 0)
Iran_strain = np.array(Iran.strain)

In [4]:
# initiate matrix Nepal_alpha
Nepal_alpha = np.ones((strain_number, strain_number))*50

#In Nepal alpha matrix, find the positions of alpha_ij already estimated before in Brazil, Denmark, Mozambique, Iran and fills with these values
for xN, strain_xN in enumerate(Nepal_strain):
    for yN, strain_yN in enumerate(Nepal_strain):
        #insert from Brazil
        find_xBrazil = np.where(Brazil_strain == strain_xN)
        index_xBrazil = find_xBrazil[0]

        find_yBrazil = np.where(Brazil_strain == strain_yN)
        index_yBrazil = find_yBrazil[0]

        #insert from Iran
        find_xIran = np.where(Iran_strain == strain_xN)
        index_xIran = find_xIran[0]

        find_yIran = np.where(Iran_strain == strain_yN)
        index_yIran = find_yIran[0]

        #insert from Denmark
        find_xDenmark = np.where(Denmark_strain == strain_xN)
        index_xDenmark = find_xDenmark[0]

        find_yDenmark = np.where(Denmark_strain == strain_yN)
        index_yDenmark = find_yDenmark[0]

        #insert from Mozambique
        find_xMozambique = np.where(Mozambique_strain == strain_xN)
        index_xMozambique = find_xMozambique[0]

        find_yMozambique = np.where(Mozambique_strain == strain_yN)
        index_yMozambique = find_yMozambique[0]


        if index_xBrazil.size > 0 and index_yBrazil.size > 0:
            Nepal_alpha[xN, yN] = Brazil_alpha[index_xBrazil, index_yBrazil]
        elif index_xIran.size > 0 and index_yIran.size > 0:
            Nepal_alpha[xN, yN] = Iran_alpha[index_xIran, index_yIran]
        elif index_xDenmark.size > 0 and index_yDenmark.size > 0:
            Nepal_alpha[xN, yN] = Denmark_alpha[index_xDenmark, index_yDenmark]
        elif index_xMozambique.size > 0 and index_yMozambique.size > 0:
            Nepal_alpha[xN, yN] = Mozambique_alpha[index_xMozambique, index_yMozambique]

#np.savetxt('fix_alpha_Nepal.txt', Nepal_alpha, delimiter = ',')

In [5]:
#find the length of vector that we need to fit the matrix Nepal alpha
length = 0

for i in range(0, strain_number):
    for j in range(0, strain_number):
        if Nepal_alpha[i,j] > 49:
            length += 1

print(length)
#so we need to fit the matrix Nepal

2860


In [6]:
#find the coordinate of missing alpha_ij in alpha_matrix
position = []
count = 0

for i in range (0, strain_number):
    for j in range (0, strain_number):
        if Nepal_alpha[i,j] > 49:
            position.append(np.array([i,j]))
            count += 1

In [7]:
#cost function for PSO
one = np.ones(strain_number) #vector of all ones

def func(vector):
    # insert alpha vector to Nepal_alpha
    alpha_matrix = np.loadtxt(open("fix_alpha_Nepal.txt","rb"), delimiter = ",", skiprows = 0)

    for i in range (0, length):
        coordinate = position[i]
        alpha_matrix[coordinate[0], coordinate[1]] = vector[i]

    #compute invasion fitness matrix
    lambd = np.zeros((strain_number, strain_number))
    for i in range (0, strain_number):
        for j in range (0, strain_number):
            lambd[i][j] = mu*(alpha_matrix[j][i] - alpha_matrix[i][j]) + alpha_matrix[j][i] - alpha_matrix[j][j]

    # compute the equilibrium of predicted model
    inv = np.linalg.inv(lambd) #inverse matrix of fitness matrix
    dot = np.matmul(inv, one)
    z = (1/np.matmul(one.T, dot))*dot
    
    err = np.linalg.norm(Nepal_cases - z)
    #compute Jacobian matrix with Nepal's frequencies
    jac = jacobian_func(lambd, z)
    #compute eigenvalues of Jacobian matrix
    eigenvalues, eigenvectors = np.linalg.eig(jac)
    eig_real = eigenvalues.real
    eig_real = eig_real.tolist()

    stability = max(eig_real)

    return stability, err #we want minimize error and all negative eigenvalues for the sake of stability

In [10]:
#objective function is base on estimation error and stability (all eigenvalues must have negative real parts)
def func_opt(vector):
    stability, error = func(vector)
    return stability + error/2

In [11]:
# RUN PSO algo
# initial starting location [x1,x2...]
initial = [0]*length

# input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
bounds = []
for i in range (0, length):
    bounds.append(np.array([-10, 10]))
bounds = np.array(bounds)

# run the algorithm
PSO(func_opt, initial, bounds, num_particles = 4000, maxiter = 4000)

FINAL:
[0.13471596844509612, -0.2474776950965692, -0.0935543767541767, 0.28617133917726667, -0.3579594598567403, 2.2850525111758953, 0.7732359871583792, 0.4385714920979544, -0.2235790982559568, -0.4977615161889247, -1.9219854758945547, -0.07627320580858958, -0.20052413365954447, 1.3611424005698285, 0.43510772099827005, -0.7733042089888373, 0.4475495803943701, -0.7269482511313525, -0.3319664497045149, -0.3403782731496428, 0.383509486733692, 1.0201144165520075, 0.6899917302909376, 0.3141952107559803, -0.19039287741766156, -0.11630858768530382, 1.054409052670246, 4.170052751571966, 0.30100494468645417, 0.5299319318499306, -0.4935174962714235, 0.13600322459639663, -0.3864571055856159, 0.7777255252547356, 1.057752826225425, 0.027046579672945485, -0.23511072304208908, 0.0828987436102843, 0.4461091783087334, 0.5440545077508885, 0.38612246011997287, 4.146309441408274, -0.2684455980497967, 0.3435544161606426, 0.5419089464127534, -1.8462494611985385, 1.1840245309747461, -0.055161924150975636, 0.

In [12]:
#the 2860-vector predicted
opti_Nepal = [0.13471596844509612, -0.2474776950965692, -0.0935543767541767, 0.28617133917726667, -0.3579594598567403, 2.2850525111758953, 0.7732359871583792, 0.4385714920979544, -0.2235790982559568, -0.4977615161889247, -1.9219854758945547, -0.07627320580858958, -0.20052413365954447, 1.3611424005698285, 0.43510772099827005, -0.7733042089888373, 0.4475495803943701, -0.7269482511313525, -0.3319664497045149, -0.3403782731496428, 0.383509486733692, 1.0201144165520075, 0.6899917302909376, 0.3141952107559803, -0.19039287741766156, -0.11630858768530382, 1.054409052670246, 4.170052751571966, 0.30100494468645417, 0.5299319318499306, -0.4935174962714235, 0.13600322459639663, -0.3864571055856159, 0.7777255252547356, 1.057752826225425, 0.027046579672945485, -0.23511072304208908, 0.0828987436102843, 0.4461091783087334, 0.5440545077508885, 0.38612246011997287, 4.146309441408274, -0.2684455980497967, 0.3435544161606426, 0.5419089464127534, -1.8462494611985385, 1.1840245309747461, -0.055161924150975636, 0.10958891249770637, 0.4273444825611305, 0.4710104103734261, 0.24982696696166687, 2.2105071992666905, -0.5226361846493375, -0.11547356583442211, 0.3972109564270395, -0.6692740312607612, 0.01911962337915113, -1.4003283370787203, -0.9235506616276739, -0.21439586146116899, -1.552274150322186, 0.3047693475988207, 2.062062655613797, 1.4059965100818528, 0.8150611169833244, 1.9773223977322156, -1.554824821527806, -0.517125877717978, -0.7304850695593337, 0.052936250804682776, 0.06821746260459907, 0.09731151497598427, 2.20608686681043, -0.3136962016742858, 0.5795629497254595, -0.4190416275914881, 0.8975184859356619, -0.062270009394281775, -0.6027740415043203, 0.1008665810005118, 0.43782351700429, 0.195602930120275, 0.8098779123716155, -2.578070521173312, -0.19440985581033388, -0.08898877025327953, 1.3137323610338583, -3.3366977701168947, 0.11093224454123798, -0.7777635654089722, 1.4187460615457372, 0.5046217014610878, 0.37972306295273217, 0.986953697049873, 1.2043176277420502, 0.14005306662088368, 0.05591867512124498, -0.631314091654132, -0.5163351262819336, -0.5002268990259816, 0.5688533315844493, -0.11704656624297927, -0.018170812666749636, -0.0023601406653130895, -0.25609444769550277, -3.03922164043297, 0.5736482578090273, 0.990964922388724, -0.342149351593589, 0.32290708518834066, -0.39991665780789915, 0.2323717017915542, 0.8475361161090316, -0.3579467346001853, 0.04292879328165448, 0.6877007114827409, 0.4960526362767784, 0.17461763034165778, 0.18214871863517887, -2.6045483408128813, -0.42211590992103715, -0.013049082487049678, -0.545859490984133, -0.6308295965507354, 0.722953961345065, 0.28273787400496087, 0.016706079349773004, 1.1649932921609096, -0.08026525647699524, -0.10899681704046668, -0.15135959223191287, 0.27422241690740956, -0.962707031967986, 0.7847413886242851, -1.083985911043255, -0.4533411465253204, -0.45829205060532674, -0.469602483852287, 0.1782546206519743, -0.08424489215719612, -1.256011735096587, -0.30647642079438825, -0.012267829508039571, -0.39060440998928037, -0.3155903868703693, -0.19055490796877492, -0.1622523455967404, 0.05708162800580476, 0.5697840070161017, 0.7112389290474816, -0.7653159357367435, 0.3338158154803334, 0.8161316719449927, -0.46962329130400093, 0.32807072293579426, 0.7119068919955769, -1.0041514691392666, 0.15675898963211382, 0.07941697709271824, 0.06153676397491663, 2.793348486270708, -0.7716035422152636, -0.22255289689651495, -0.11886915218600574, 4.8499633019571124, 0.36738937712344705, -1.1725710597080052, -1.2810084057334092, -1.4311704877435634, 0.5932207358207299, 0.3016564370360805, -0.8958877541846129, 0.21457702946387472, -0.4439061970867088, 2.667740495342813, -0.18376892115569785, 0.8336389743970577, 2.017884934485556, 0.039998633031666696, -0.282212900457506, 0.8142237670161598, 1.3565903547954088, 0.3121774853957687, -0.05108421465154288, 0.9501990127959088, 0.4418146768276816, -0.491009180696739, -0.3150686736697019, -0.8370933211769588, 0.24773495612251623, 0.8282325321878535, 0.5343124580973547, 0.4105348105202297, -0.9664412765153048, 1.3053877695076004, 0.5375683093322424, -0.7034082287727452, 0.3635552515127342, -0.11020083737197292, 0.6457159318312622, -1.11155241753209, -0.1531240097251167, -0.5654425067463863, -0.22107315126457197, 0.16799278948700908, -1.1512249680701871, 0.3218036874632711, -1.3834541167094392, -0.6268218247063461, -1.410489963057997, -0.7782206726307409, 1.0351412299505203, 1.0506876034776675, 2.002178359808059, -0.22868476967562185, -3.030869075641595, -0.2552790177613426, -0.8085008507674307, -0.0764870726949074, 1.0002663908768559, -0.05466476925137481, -1.0743899455268, 0.22107181685677368, 1.856436747882955, 1.64531147090269, -0.49900372090898965, -1.018636382169533, 0.23059881246396, 3.496302831535773, -0.5300641024261904, 0.06105081787858387, -0.9141032541815477, 1.4743961419094491, 0.9963532235923948, -0.5794795136608186, 0.32303650914134585, -0.38821892847027567, -0.0850338653449838, -0.013101079851372602, 0.042955315858871815, -0.6589604572147891, 0.289500151141356, -0.48751183259238906, 2.285199579230659, -1.2078837923866872, 0.38055471270649227, 0.5321306462310594, -0.42640839183570717, 0.43787585814926416, 0.22546007502967938, 3.1499960042620327, -0.7835045337097716, 0.9046345896355146, 0.014439175128836692, 1.0272827339259327, -2.095752799100653, 0.07234500265295547, 0.10975495740310233, -1.0029422898696083, -0.05919622649763694, 1.405068620336552, -0.7090600458401444, -0.24078169955002393, -0.38072011305732845, -0.02452461563257701, -0.034576493355648455, -0.9564258710233937, -1.6618360545678432, 0.033137631059648204, 0.9771123841302378, 0.5098316278539977, 0.278726247407615, 0.5031728360618425, 0.16760558511679893, -0.05603698366692489, -0.29549919720166573, -0.6956421217305667, -0.05890190257289333, -0.837995054171791, 2.3354677652094464, 0.009222109044129644, 0.12293764059555175, 0.34405053670435315, -1.4771849417293477, 0.04634608747170887, 0.20256347232999408, -1.4018826298337685, -0.6649109449076669, 1.0307312374376976, -0.42748359205985514, 0.130318865135529, 0.7551488182513071, 0.4698675079007067, -0.017316341558130215, -0.29484633297988, -0.011975790183534975, -0.07537864054511467, -0.38281716217340844, -0.28686486076754075, 0.11582197429641201, 0.46248263133714734, 0.041553601049653256, -0.4491352911944564, -0.3205005870826675, -0.7745785764844912, 1.2634114921824908, -0.49433750916563046, 0.06932688736147731, 0.020386894563592193, 1.0005692315867925, -1.8990991433208113, 1.138280231160438, -1.1627322091195464, -0.0990046989399255, 0.48839210209887296, -0.34665668612898903, 0.5372203807434102, 1.2973960559685587, -1.6927891314787826, 0.04711907932263867, 1.3325986780682009, -0.1398542455570588, 0.7739868223885589, -0.3535122998977978, -0.15421536885682227, -0.18999862657674554, -2.866291481498127, -1.3177309947633293, -1.4000407776357355, 0.490178017689456, -1.541091353828305, -0.5117860144940815, 0.21543163537638685, -0.44965885762352925, 0.22616962947120278, 0.18547370516999062, -0.5950803112074006, 0.936448122521265, -0.2809768533794643, 2.036709800425151, 0.11202185932663009, -0.4016784006961092, 0.052918855511331804, -0.6682236644252214, -0.7642711140819, 0.02146987141983399, -0.3573874567084905, -0.22359656977234024, -0.7461424888237317, -1.2667296613898402, -0.4471292625478302, -0.21051484634298595, -0.5358763420580845, -0.5323440171418548, 1.1600089046251059, -0.38130070126277776, 0.9163144749600586, -0.18082872710950168, 1.1575764854465873, -0.2013962479802009, -0.49127921216465203, -1.0165049079767152, 0.523475476396235, 1.4733297767005498, -0.031107550797743085, -0.4419124920488243, -1.1264351276775595, -0.24005505245600753, 0.4700060029013066, 0.03018461709667932, -0.1360655262361409, 0.2272807605681509, 0.14164782871335177, 0.5254245669077305, -2.938384668669822, 0.17220918351248238, -0.34205192066253043, 0.371155769820714, 0.18880331663372948, -0.12432721687355307, 0.4875912843984961, -0.5674518022228775, 0.9532389752029182, 1.6051441433369262, -0.08151017366692717, -1.6296350850090409, -0.3228036806543188, -0.41632381952855885, 0.38355734009093134, -1.2409893324449748, -2.8123051414314553, 0.7747428044450975, 0.14775716285723464, 0.24258723643659094, 0.7808067144713655, 0.37330451279164373, -0.6227580035111773, -3.090850793905032, 1.6101021926926613, 0.25314838347545937, -0.6346406887297219, 0.06846534273950453, -0.5687891038535771, 0.8901265475208244, -1.0699897427038174, -1.6587529113198312, -0.44753378209870043, -2.465140491285842, -0.19639534803151393, 1.0034523392984258, 2.7345925099415243, -0.6066330448719897, 0.5454203889590065, -0.029118672239430248, 0.260680621845869, -0.3054909321506528, -0.612393657985516, -0.0004401724002616498, 0.9715347228701152, -0.6596878003079628, -0.22277309469736584, -0.9222312783147937, -0.3832231282497155, -0.08226716661134457, 0.07983604104730913, 0.2958718207921035, -0.07882952680976585, 0.48445967577635507, 1.3079770317630932, -0.9262623492160922, 0.9733386013517044, -0.23107025229647785, 1.3202628022183305, -0.29107476194704085, 1.1317849489946545, 0.09004359619133036, -0.7732640207343472, -0.5276476848182315, 0.670078990996675, 0.24209604099668708, 0.8195039447746795, -0.7071701209243656, 0.6653692673018732, -0.681361680843546, 0.7135532842363779, -0.5109048639848376, 0.11457313172184688, -0.6126216335104088, -0.16329789293839078, -2.7195240218789225, -0.9540910025867707, -0.4141581846107016, -0.2700499246588361, -0.7034191880693594, -0.2976553474346303, -0.6238376125725918, 2.581075966971419, 0.2899676865412575, -0.5524769788588014, -0.6478399711240604, -0.9401464879482397, 0.25557178075065046, -1.280914211304387, 0.518395869557638, 1.919162711840855, 0.035586158198386474, -0.024528076264765827, -0.2856149423751383, -0.37643552610511294, 0.10247230919644688, -0.2490680739582778, 0.03392937916117919, 1.5747643552106088, -0.46571339721849214, -1.2032836986951885, 0.10918193138723213, -0.5119172124125373, 1.1221366916599966, 0.17304851025047638, -3.4432565763461613, -0.5634003799261347, 0.346297996569814, 0.28541688198399895, -2.4601032240365908, 0.3389582289926073, 0.20025792207953527, 0.3274249057100962, -1.1979836987919052, -1.5560314431497277, -1.0858999462921841, -0.6466768284269507, 0.5377936070756881, 2.3113944980676937, -1.5297137254585966, 0.2508356218044366, 0.3727234871116253, -3.2605114942768068, 0.8062207457901083, 0.2953562328442214, 1.0163579811822503, 0.08836864751172759, -0.7134759006616024, 0.9938305157675059, -1.4592293213045526, 0.6573741833593837, 1.0242127036757083, -0.06456711768242392, 0.3193355914276321, -0.013797771906009698, 0.06435426980597293, -0.9317537518702008, -0.2767411996803685, -0.2218752204508491, -0.14440278217235622, -6.865039833056831, 0.3990196469859816, 1.8374628845838197, 0.6069988926580685, -2.734572075624247, -0.5038149519752045, -0.018414353335102586, 0.476589424290814, 0.3001944400777791, -0.1679895561493022, -1.2059119416754644, 0.4209599990610267, -1.7052822126824911, -1.8464495789462638, 0.12381301479771209, 1.5087428362131514, 0.5299122375150628, 0.900188938054253, -0.31782789213510637, -0.5155074447992672, 0.5923703966281758, -0.38544349042145715, -1.7905788716968343, -0.38301995741319117, 0.1976059576293736, 1.6776097026909977, -0.36054017605104777, -0.12952432305315187, -0.3783157727000618, 1.3902766201224825, 3.4408292031566012, 0.22894138528981584, -0.19926483414792154, 0.338708520160007, -0.28228925473692373, 0.8091534250649882, -0.8064044159116764, -0.44944611513437005, 0.09037822369438314, 0.96189574396521, -1.1031712574111112, -0.3602986457918561, 1.8677135157400604, -2.7880215400779584, -0.014773998514700374, -0.062002182265172595, 0.30494099834167865, 0.02355047576558089, 0.27975265857137666, 0.9772258658475405, 0.6640638199862365, -0.8440766232101321, 2.89198628664033, 0.16090470099527346, 0.5835633485716543, -0.2648548752712196, 0.39769319081470367, 1.1872017140465698, 0.4513310492117811, -0.2858718152397619, 1.6941220455633503, -0.9874605347378983, 0.5664866486513753, 3.6808366122657104, -0.09998339233791578, 0.5893448629376412, 0.6042827371639683, -0.6421428947739637, 0.4372312645191118, 0.46031366114092703, -1.0790286894330476, -1.3656400388656165, -0.47723011390260067, -1.3609543254614271, 1.111240512436544, 5.115552355568695, 0.880097972382516, -0.1558505202421791, -0.6288738647282427, -1.0802246081909195, 0.8158749579325151, -0.0033001958720825257, -2.905659381249321, -0.2973826276223575, -0.8213832562598796, 0.15017788646730784, -0.10449432739217253, -0.13207069579198313, 0.26934112774711344, -1.7554988025950327, -0.03282701361074725, 0.6763274912971321, -1.6509504408677398, 0.4841892990337162, 1.060119936046671, -1.4524733874554778, 2.2794098032279395, -0.3554352670117083, -0.3927543155532771, 0.02641616077264524, -0.3760765274898328, -1.9578212207670171, 1.6534318364164116, -0.3059730479897975, 0.2497073519522182, -0.4725040938785144, -0.3814523054177586, 1.0648884259524591, -1.8253251542437947, -0.3713745985848471, 0.7561447185396507, -0.558853258190675, -0.05590388879823643, 0.9825541461795293, 2.1070991866011832, 0.14531688315271657, 1.2388573169529118, 1.0948726343091648, -0.16417690824549444, -0.38969334198700606, -0.15016956756247324, -1.4278781375043357, 0.011712211732169877, 0.18111055635449302, 1.515828853079666, -0.5977421207475561, 1.540440141440046, 0.321390345011604, 0.3321630664576755, -1.1649377485987003, -0.4043511214802738, -1.0055988314739954, -0.14410554932030412, 0.26164922411388114, -0.03697237946100289, -3.8078401045012407, -0.08178928729103943, 0.9555882727551288, -1.1448494656114672, -3.079963790006289, 0.3548356817209628, -1.7349057253419158, 0.16073158555165026, 0.025257821701995656, 2.115458515011788, 0.14200431892830973, 0.12891581037168853, -0.7872897211119683, -0.8390158911837764, 0.7589430410091073, -1.1108118878205966, 0.019861191890375454, 0.09685044219111712, 0.1511614447864769, 0.22067077714740407, 0.5222810635804397, 1.1426146457332893, 0.8294209248785392, -0.6778989288662224, 0.05991234529044138, -7.2355822220058235, 0.004772420942531937, -1.0112947987169827, -0.0836071519136656, 1.1485308597608381, 0.5333513636089028, 3.1409267135707606, 0.45282782889569057, -1.6535637571002908, -0.27075229270484835, 0.12605483487139754, -0.4078290667857452, -0.14435680158926167, 0.48959571609144065, 0.3366974702984647, -0.2671534867947766, 0.5594646519705336, 0.6322637185919547, 0.06371998466126493, 0.781937103787218, 0.5870465847875597, 0.4440204897080405, -2.387439659094579, -0.519520917576227, -0.01690228090045211, 0.4068864072110229, 0.33852559053233994, 2.770201008635723, -1.8160681085908337, -0.8989286433405514, -0.16509952434986927, 0.15857816823195475, -2.8788300619310156, 0.9954992382809994, 0.8044095804553826, -1.152891761162508, 0.05637644956570675, 0.2762940132479817, -0.053405567383917676, 0.14131089417021747, 0.19289993847911938, -0.6680286471756407, -1.155642150602419, 1.9574228967529788, -0.3738356577197375, 2.3658623354667103, 0.4988341554720479, 0.3439982990615149, 2.1302971138670346, 1.4427276732798515, -0.6424088760354438, 1.1878186107843869, 1.2197529828111042, -0.8928467925093111, -0.4388171041669913, -0.4458884130997274, 0.6701871005600021, -0.635296683675712, -1.2829588007490973, -0.34392427649684604, 1.1331307796263232, -0.405571599551067, -0.12549320747945591, -0.9452400971757167, -0.3642662005910147, 1.6441766364206878, -0.8474663987871232, -0.08308855969634515, -0.35792204091459684, -0.7913316607826122, -2.215212051748429, -0.17534123732466797, 1.0675080216325386, 0.9818762815326001, 0.21655919274974797, -0.3722731478831871, 0.18215739934536257, 0.13953051732878885, -0.03899736977786813, 0.3560964859957892, -0.20900410556213617, 0.2836877538780798, 2.975456395969614, 0.3708173276413322, -0.10742443583143628, -0.30766365454382777, 0.9279627193849341, -0.5585442148144907, 0.42691045657887405, 3.583449584821368, -0.5256511840231801, 0.09883986511554108, -0.8152955162996128, 0.21913795804522065, 1.9535403253382768, 0.4359111695555031, 0.3031098138587052, -0.7050561443535978, 0.6746097130412294, -0.7820049770928527, -2.122737770997362, -0.963084514141055, -0.487138508192546, -0.03528900498079556, 0.3473169678707928, -0.8747829134428444, -1.0067504286642894, 0.8276707813017536, -0.9667395794177516, 1.0505305336298798, -0.04870354248988012, -0.49035437340029364, 0.14646798647539164, 1.921083453124673, 0.29332116189210883, -0.2128969527409715, -0.10222076493819744, 0.10228639381062858, 0.21798582338822106, 0.47081382162531693, -1.171197015540455, 0.2623574371048425, -1.7833763816531898, -0.11735220222773067, 0.6867615979450876, 0.2913552270164527, 0.9824387887344409, -1.4631435210093322, 0.7223158809865161, -0.9578354110074752, 0.905102288130378, 1.5063306795528422, 0.038477520218724845, -0.40084563621136654, 0.24100769277603815, 0.07415654737733664, -0.14062146573298961, -3.4974839079315516, -0.5212021921210532, -0.9858733943044339, 0.7756613716785925, -0.35908897202550044, -1.2505138557884548, -0.3911501195378304, 0.9301371172448378, -1.7945713756347876, 0.10796025073340063, -0.25211237807397013, 0.4250985450386592, -0.22843432507671702, 0.38343211028703517, -1.7097436006787792, 0.7148189639212011, 0.08939196135232592, 1.1185022257497204, 0.5559482082348534, -0.34900329572044697, -0.6636439581548399, -0.1909885243754318, -0.03874788168199846, 0.4219690931364662, -0.370439742067482, -0.026860321400546817, -0.5363849542089216, -0.5944170248979853, -0.12117897634366072, -0.5801169072737015, 0.9765160440397538, 0.15679528302164078, -0.47338444555304204, 1.2610876861320843, 0.1295453752245277, -7.931089949904301, -1.0658002112081801, 0.39363152141322033, 0.01315110951594087, 0.4085198312798843, 1.338482334685298, 1.0837849667827648, -0.3688902745496983, 0.27665619429184674, -0.3620200543821704, -2.9570777082370086, -0.15186761887017386, -0.38244873328731255, 0.16131739463207886, 1.6486804110150606, 0.2529448871265271, -0.14377471232714517, -0.33817014923742555, 0.4640688301846337, 0.37638319870963777, 1.0870870421365624, -0.7601269686925826, -0.3942849810828575, -0.6293949573707212, 0.3846412999016487, 3.4802524127044703, 3.312259706930117, 0.724105207052856, -0.37950360861491617, -0.6806612434647662, 0.5965300710406494, 0.4297352332429088, -0.6301365266784507, 0.10861452786831045, 0.4611123491285035, 0.9139543774801763, -1.8974826908471665, -1.2436409868350105, 0.09978072931548389, -1.1666571700747985, -0.008640184297487835, 0.17587152529326222, 0.08520577726727258, -2.186631236005925, 0.1464720017725591, -1.0552688157996177, -0.26010577665830503, -0.5691336131419877, -0.11552025708862138, 0.8911556691574938, -2.010293096070126, 0.040460742421697574, -0.2735084922717689, -0.07555344535470643, 0.5421438975484557, 0.18788425222250751, 0.9105979032214222, -0.47986330713552366, 5.954800782083113, 0.344757060593846, -0.16051888566149589, 0.3348409873597828, -0.0831481162356149, 1.0236495629654156, 1.437708775837405, -2.828818183049532, 0.09022983927566657, 0.24704889705972174, 0.1575090954690319, -1.176922681763087, 2.302642939533925, 0.1830256311126088, 4.836729051692841, -0.24669643509792324, 0.9026087601921955, 3.226290203350099, -0.5298787320240089, -1.5511007145277003, 0.168505874126084, 1.5605112086285517, -0.5547300146155708, 0.34241284429427365, -1.3391710967577803, -0.7985040870309619, 0.4769676814163153, 0.13731543381182584, 0.49454179936708936, -1.5711484048635915, -0.027445333404550236, 0.21110667051536686, 0.7634570781297852, 0.26311285636061077, 1.4130175200245965, 0.5246478764354295, -0.8509301808660625, 0.2317438514515153, 1.1467607281289887, 0.2758992616531991, -1.4174655683294575, 0.20555630591884944, 0.6922786436067122, -0.8909798445460876, -2.7581917355900742, -0.1302072828663493, -0.0995093977587889, -0.5021571872892532, 0.4693159975347678, -0.3240003851982518, -0.08186852942430867, -0.1989317305517504, 0.13521090515266312, 0.23089055772627923, 0.7245162981058332, 1.2561501906455121, -0.8228416443612804, 2.0692878627823195, 0.13116439741943134, -3.6135919372600873, 2.5013934531516027, 0.8420045663600226, 0.15570743753400854, -0.620055190885552, -0.4169494518288695, -0.06959333037136799, -0.1402832804176138, 0.4456301354944097, 0.07328534094121987, 0.25564001263536384, -0.13161649884447676, 1.4856616396251834, -0.9886198581078358, -0.34641181187694425, 0.29193579550536775, -1.1331373355343812, 0.21048427247142792, -0.7623582397285551, 0.582144788269172, 0.14219311262434492, -1.5463584450126953, 0.4643207467906742, -0.1857858619889001, 0.15461120986258753, 0.5459754854699228, 0.3836530204501989, 0.4869642970017613, -0.164275095938989, 0.38495547667713503, 0.24789251465608889, 0.22666847054679443, 0.32959432162436947, -1.5155076083035737, 0.31368523208581833, -1.136719289864364, -0.00013471209082228193, -2.3942225864138207, -1.2425859795568492, -2.1010726233767425, 0.5317703426114071, -1.822776044681277, 0.2710520950873459, 0.33638004045927344, 0.3279793195955848, 3.335045232051333, 1.187050208851497, 0.11845643079784146, 0.7175540549700873, 0.37484714948998205, 0.2150417984665901, -0.4362201167643906, 0.14211979982288778, 10.0, -0.18311480656358167, -0.31095021774796067, -2.2298722745651083, -2.994902919235062, 0.7486423383673402, 1.5135018554811992, 0.38814997039927196, -0.04582690561637971, -0.8193618337901196, -3.3022532110698606, -1.4227790998456051, 0.13400334538996742, -0.45857071843656744, -0.01569839446895203, -0.6582365486693587, 0.5584749383891624, 0.2755043214037263, 0.8196458200650704, -0.22988477395620094, -0.39983945042593133, -0.22764581704900078, -1.40420631072548, -0.7290186989105853, -0.11335015160143763, 0.025251509228880573, -0.5199061517300039, -0.7752949199299197, -2.0573128522621413, -0.8995794884028339, -0.3464823276984486, -0.7842628695066323, -0.1384955970116889, -2.165988873019952, 7.632816242406823, 0.5041642324124951, 2.705232219249699, -0.09045657856689227, -0.5578749000976713, 5.680578689679916, 0.1195791343696949, 0.15076250934691487, 0.9612169522130855, 0.32354733766718996, 0.43873787555148347, 2.168094680483717, 1.7389767807657566, -0.7977902993270899, 0.5215779768626351, -0.8073131817191262, -0.32552557261571846, -0.6359153727420798, -0.1348131695344662, -1.0633666450004005, 0.021677060383081457, 0.7421138869569022, -0.6580861682399767, -0.11133426510432211, -0.018873533951782054, 1.7293307366905908, 1.3428908598062013, 0.02974218184610219, 0.6641603415428996, -0.2569882442329066, 3.420774724528858, 0.7006644435531306, -0.6758526292604885, 0.5082121156773973, 0.9855374856300778, -0.020337062595366898, -0.8691681001805264, -0.9915586803242984, -0.13499671633570504, 0.24461667831167075, -0.40432812514462313, -0.9308415236533302, 0.6510095133640467, 0.7045713456821062, 1.2963475702610088, 0.5634900895767523, 0.6219072536205614, 0.496402130966335, 0.152397737135186, 5.196039916032126, 0.1000283668559629, 0.12385593149076717, 2.2127270211027854, -3.1734109472713112, 0.4161237151956014, 1.3575328260571424, 0.4094230453667794, 2.8121255758978756, 0.6687404187065185, 1.1734159966548134, -0.4215291479545712, 1.09471482518085, -1.0056306444586374, 0.6417176939530512, -0.011254717074773385, -1.4817659244260573, -1.95754132908268, 1.1324743698811361, 0.25293438999681195, 1.1998328456889655, 0.6367894173205975, -0.32930671619497237, -0.5615174726237373, 0.6999526858740135, 0.7070332808328404, 0.5680576271137628, 0.7103164416128751, -1.5166904621995774, -0.7155887368319375, 0.3568747886242262, -1.037554408569109, 10.0, -0.8096767015051983, 0.620521590700037, -1.2451007759686425, -2.283335386101095, -0.3794386801570839, -0.007548152332071718, 0.6491862648901249, 0.011469475194373906, -0.8595528674417957, -0.5258811634294288, 1.1177332154029525, -0.42040083877060713, 0.2669375441092222, -0.24880969414137522, -0.3050693704719392, -0.7248345656793596, -0.8469965888891247, 2.223834594555115, -0.8195254134039234, -0.9955920962372523, -0.05376074255623459, -0.2560939293622119, 0.8691293678919844, -0.17141652499284357, 0.0652551593486558, -0.3988631494908359, 1.4664021135923093, 1.0006626733171398, -8.705436111652558, 0.2358240328958932, 0.49432378700178425, -1.8297660172258354, 0.2776011159218072, -1.0415777384508509, -0.2499809036475368, 0.06323261622264331, -0.7020628639374258, -2.1741659930885944, -0.16134387345000545, 0.008275905456285579, -1.5880582778090524, 0.3326419525205473, 1.6014511141372743, -0.35710076752784337, -0.1497592368601517, 1.1304909567411299, 0.163306115720058, 0.628421361653011, -0.14149862760078846, 0.3631998026783551, 0.8513190630641099, 0.7476014122586673, -1.148136281787666, 1.3794404584450044, 0.5994829734144397, -0.4841175518091642, -0.4022864645738281, -0.016100876354382603, 0.3349428352796559, 0.2889476277139133, -0.3852061580332898, -0.2770668924311196, -0.20296046993334696, -0.6548255699670918, -1.089104071714024, -0.3965719467529187, 0.9396778470870863, 0.39029321197018174, 2.4523629063287316, -0.4717494072445452, -1.3874762003336032, 1.0925773610190324, 0.5102450906408257, 0.33193700830370126, 0.17901771187761759, 0.47257012161662804, 0.2421651934969845, -0.13486527291963332, -2.5947166670613617, 0.9940068881804521, 0.012063249223149804, 1.4146850107550895, -0.19148471880406495, -1.4573987309331216, 1.103588524474691, 0.18797525754878164, -0.3125142342536558, 0.2853377191463632, -0.23368053833238223, 1.6259592028762406, 0.4497879511395176, 0.04358308602663021, -1.7831599162075353, 4.6685526149875285, 0.9864959436308773, -0.22088289794293667, -0.4975853427769111, 1.1526169244748345, 0.6010914787071014, 0.48612029379962457, 0.824963306347592, -0.4777083118315937, 0.6084077883801157, 0.27973420280858446, 0.7535225895508256, 0.7740897759936394, 0.335313429672742, -0.28660751382537586, 0.2922001856775672, -0.2840020769230705, -0.26335840953316786, 1.7076283057465467, 0.35750608089140457, -0.7061417152788918, -5.451664628531985, 0.5246397349112168, 1.5304408600898953, -0.31117003623729006, 0.2064746771680897, -3.026942896248663, 0.03898065655246631, -2.727743902530415, 0.18537654093301237, -0.6779260345490552, -0.7955852815044564, 0.7146025657434413, -0.3618767463580727, -1.771166797263096, 0.045188256897900056, 1.2125538669876563, -1.9887970345828057, -0.3202927089060245, -0.4364978338099323, 0.6118090890105713, 0.6511042347557792, 0.8075877548623425, 0.37418695341102926, -0.30355992339547944, -0.5868226037598332, 4.855158014833153, -0.09015337054292046, -0.013085762542310956, -0.3877860595680135, -0.10854323221950486, -0.29866076380296996, -0.7503854628913477, -0.5412572869676435, 0.09769294808807676, -0.46642213937084165, 0.01547911631438348, -0.23856152372794423, -0.38147726428848994, -0.797680289601369, 0.0618021659676798, 0.004791113822153, -0.19849477324152712, 1.332784104728169, 0.9498762830243197, 0.709317495723163, -0.10333979699383194, -0.5651083061675359, 0.16347595806506582, 0.8349798319089374, 0.7088556233231693, -0.5447163510277176, -0.3513305820595722, 0.015588685840353963, -1.1085712537152057, -4.619212364117906, 0.1172169521100864, -0.14552402204354795, 0.05854912935998484, 1.947638677371782, -0.3286079301551408, 0.10303697217480966, 0.3194857081333385, 0.0719393467785416, 0.791923923970701, 0.6946026711354887, -0.48448428817206185, -0.38019577396795945, -0.14781753163059272, -0.040400390742236274, 0.0077676330409872235, 1.994311218861343, 0.1599029332054367, -0.1501685593291814, -0.46403253407992423, -0.7681779619577065, -0.057727115020595555, 7.707232625276187e-06, 0.07821650330560956, -0.6804485117243129, 0.6277041309465917, -0.9929035138055055, -0.22331254802482461, 8.247422012631644, 0.10283907649137239, 3.7832254673469485, -0.16439091306895992, -0.5889629901355389, 0.07679195382101907, 1.137071244160721, 0.05379484369360128, -0.8856414804487877, -0.21805017859538006, 0.2690881030844647, 0.12492179678480646, -0.029510812449505208, 1.2399948106292427, 0.6767942185620881, -1.6955544737838444, -0.6040374081921251, -0.2613155350613715, -0.4902800298924818, -1.1666208881264764, -0.028603889425645256, -0.11580260910969112, 1.41519782634525, 1.1366107447167497, 0.17090938154063098, -0.5413395632123648, 0.4368971226327504, -0.23612525563122613, -0.4855848354464349, 0.672391926233722, -0.6506014228516697, -0.7118287780543504, -0.7551988761661992, -1.2895991763153383, 0.4365408613106166, -4.211237591418742, 0.016349050018434688, -0.9288657050454966, 0.03167100295099031, -0.7390871190083387, 0.2391915380759192, -0.17676970775204112, -1.1098839634228739, -1.5161866745655541, 0.13125740608826966, -0.33077528367036224, 0.3832740607419938, -1.3494756418613192, 0.917493202095653, -0.20280228099643488, -0.7148063683381678, 0.04040024285789193, -1.22019413484408, 0.15729245214956972, 0.19036002866764243, 9.996841880924286, 0.0707269696020613, 0.05890379265808338, 0.1636692678611331, -0.23174503445449052, -0.29495492158571807, -0.06868191943396544, 0.9173350618680938, -0.3260808058531565, -3.444067921721781, 0.1437748863644793, 0.7060592951536004, 1.6433630667722128, 0.4265708234367733, -1.2899717413219804, 0.7134277196580753, 0.37206468856775227, 0.6101801158908193, -0.0657332296502797, -0.3622954609179475, -0.3270045152110241, -0.026180479093095326, -0.5891584586533247, 0.3312156999737628, 0.2678537654645356, 1.5377853142784625, 0.31010593385425844, 0.26663730599680796, 1.2835505247919046, 0.4777596903230902, 0.14379975402331735, -1.4118378715227422, -1.0668683809283799, 0.9832802446215088, 0.10699187105995887, -0.0248426081349529, 0.8505940044856357, 0.028459242796619735, -0.8779101035325824, -0.39104338758000967, 0.28095225829422277, 0.46047229734716966, 0.2739424379458218, -0.20967201327665835, -0.9036667002417216, 0.029096584048937117, 0.12499775817263904, -0.4512595674330188, -2.560188652795387, 0.18843403038991471, 0.6726873305768059, 0.7004650887704628, -0.5162639764277914, 1.0291032552108088, -0.6582612356796249, 0.6458333859112458, 0.9899290896934579, -2.7938285023026133, -0.3366626112582278, -1.8949536474377673, -0.2708742355881119, -1.5858521217833572, -0.5121505044608496, -0.7952220981656011, 0.8813422110140812, 1.6965408501571813, -1.1076027316793577, -0.701502275451536, 0.27843337033162363, 1.5788837340717015, -1.596896009334767, -1.2078234944593824, -0.9172856308500921, 0.5388684942408011, 1.3908558957776989, -0.4371862337452461, 0.12327654981010958, -1.2366768405237982, 0.4496177967926157, -0.1578302799898699, 0.7477747691177387, -1.628342349729392, -1.011098857388325, 0.9701079051626357, 0.008097558125625112, -0.6178748285584744, 1.0120873481403765, -0.17728717775461691, 5.2149852912347034, -0.4411520749793792, 0.9971750620069977, -1.1357408978032453, 0.1871231608311874, -0.1123860878878525, 1.572148926067823, 3.904127132144797, 0.5695326950554986, 0.059163047983508577, 0.2783625183032628, -0.9919222187388262, -3.4075520463158013, 0.006494995259841869, 0.5918833921755349, -1.5786117465303666, -0.6843530818200024, -0.5898987159066781, 0.338539028852004, -0.16542838749609864, -0.05171085977224418, -0.21461415984993945, 1.640744299276523, -0.11921858525899086, -1.344092508978223, 0.17036634432711623, -0.1700402093711164, 0.9643826408906051, 0.0034971862485788986, -1.338674719994004, 0.09641429608300009, 0.1738728171231641, -0.06629999591878434, 2.560608554044097, -0.1757496409961677, 1.5751249699981666, 0.6850707724631654, 0.06957392948390184, 0.34361288119818906, -2.5498350403410153, 0.597807951906561, 0.320042920094713, -0.1456011471786009, 0.06493348889845865, -0.2845813633986802, -2.8570770135572126, -0.3900021077014027, 0.9097019704292734, -0.36728948380045834, -1.7306072153026333, -0.5782176419697133, -1.58071487019202, -1.2282552473711332, -0.5302022636131295, 0.7566973283310994, -0.041877882166579394, 0.17876999309445923, -0.7591861435265874, 0.5376434165367046, -0.14243325234622187, -0.036663203644312865, 0.1417317991235694, -0.20920044765308088, -1.4807349433997365, -0.1548818577831437, -0.3910191088411118, 0.4709793386917242, -0.3826369891613278, 0.3175939411445362, -0.9696526950814796, 0.570269247476884, 4.191590507734295, 0.0664907566399528, -1.1955024473604092, 1.421631380282536, 0.003930112746823265, 0.3562200409746011, 0.4356417726906868, -1.5024823934025453, -0.10505702561988182, -0.5681510331734351, -1.4950139417062738, -0.33442845041621266, 0.1991139901632894, 0.5345862172866251, 1.0576435757149658, -0.4407666659524575, -0.49079374727393876, 0.5504925611274839, 0.06850956052415755, -0.6678048344721415, -1.616479083825768, 0.5137208392817025, -0.12604151887034776, -0.48927963085877735, 0.4658999757397979, -0.5717297984869419, -1.6387410002240022, -0.7629387810210488, 0.17075267414405063, -1.385192940561618, 0.11824626422911681, -0.1524446306638801, -0.9532081584724862, 0.2484476782434295, -0.47458557701602566, -0.8531399969597007, -0.10984220177337933, 0.2146082150564446, 0.21648369679741863, 0.11501814899927312, 0.5166564363985189, 0.5285963215012762, -0.9706206269127635, -0.023208215283121128, -0.7736475459551184, -0.3212685901010443, 1.2400327156860538, -0.214213389991946, 0.19541679141807483, 0.060997973906109615, 0.12101156426516642, -0.5801557714579139, 8.056004007374085, -0.5401976083453711, 0.7478503645536443, 0.17501185246964104, -0.06140320212349667, -0.08371675559613051, 0.5514536260374575, -0.1044013525662864, -1.2712699338218154, -1.2331553467938952, 0.6176981186419508, 0.2415550266909552, 0.5414034636198527, -0.08951202457927532, 0.35914283139884323, 1.7975954781048464, 0.3640802691056638, -0.18928737174900662, -0.09005883787551795, -2.364872345161986, -0.10098412281868913, -0.24557892219728736, -0.40981568597118156, 0.11498546221851963, -0.12924379795284202, 0.2108172116648464, -0.2412706402857881, 0.6896225362887997, 1.163328854936957, 0.2828214280250541, -0.3334631773321617, -0.13105731947979482, -6.940008154284481, 1.527618276954324, 0.11069622703061019, 0.2798665302801934, 0.15417974066493048, -0.167212075841937, 0.2824380197310051, 1.263965399184615, -1.4216790493673899, -0.04401362494619376, 0.14688884165945115, 2.297680223668827, -0.32292186593638733, -0.4732012828964041, -0.6703376902355652, -1.1181115693698347, 0.579509090713981, 1.9917933015416103, -0.02039474016906232, -0.47935202279876804, -3.077719210613945, -1.6700697137677871, -0.8426504840451297, -0.12073637997611934, -0.12615099725971202, -0.43088272418208035, -0.8579573657845778, -0.24693880120398237, 0.21636873103623716, -0.8199318484078691, 0.1131542467284666, -1.1939072757362672, 0.5381432416092631, 0.01592664412110115, -0.09664368974846066, -1.0664644030137769, -1.0931008916881733, 0.5259794173847387, 0.28327681917157066, -1.69486753916695, 0.09425711175817718, -1.0582299500472532, -0.7496804697413275, 0.4502940201166179, 1.2920032426786197, 4.165999179188251, -0.05538971250090717, 0.405094383710447, 0.605924803234458, 0.01585142451683064, 1.7817640750479902, 0.8868020096870338, 2.317544713183456, -1.9655584161905597, -0.48462257467242875, 0.5099632720856813, 2.5709714746878047, -0.18558189055659285, -0.06477335461773326, -0.5269454004272042, 0.23316662713401515, -0.806628659971992, 2.5713315876096807, 0.22028253242690837, 0.35012292426641045, -2.8326817894568004, -0.8378102448774044, -0.35817225587809765, -0.5463566956872137, -0.587098601525286, -0.015731378165128308, -0.591107576319168, 0.9758751683442998, 1.4934404381006419, 0.27368658452047595, 0.9668011928288588, 0.9210439652193544, 0.73840432278186, -0.38530167951313726, 0.6526854899212314, -0.013199570125314128, -0.24541857926566893, 0.05269877100554376, 0.1500027533419897, -0.8565856058875637, -0.043611085077622916, -0.6013811782147797, 0.30020202901149323, -0.1354397672370173, 1.539565177267878, -0.00824305416944236, 0.40876855638944376, 0.4949498218297561, -0.9407031824954553, -0.34331335870553353, -0.02391291666800934, 0.24188380254387642, 0.7049785463965683, -0.06498471843856128, -0.7776772940955166, -0.5444492551342908, 0.7660177341711054, -1.7029201202232265, -0.5052758070189477, -0.562059864629975, -0.07715082300807334, -1.7277768281389454, -0.03403901060686359, -1.8284816490137865, 0.18460125359203905, -0.7686004217875259, 0.5079054876159629, 1.3686791722363347, -0.023316460445741274, 1.6212952884077783, 0.4276495588813279, -0.15231292427438786, -0.4030823799672279, 0.20468248797446295, -2.4103127688558104, 0.716105251248456, 0.6743692340993424, -0.9895717965040691, -0.38183424125893073, 0.9881469264982704, -1.528572066827259, 0.6172168925049585, -2.2972407435851565, -0.038663273763720016, 0.8863811399006217, -0.2845086887963708, -0.13304574048606405, -0.6775101778213999, -0.396383874251861, -0.4363572826550472, 0.2863958188774687, 1.005438146697041, 0.8772120976320795, -0.9406265283229216, -0.5347514088820455, 1.411966281870085, -0.15827298631960213, 1.2341613737593669, 0.689238996490856, 0.30446234652466364, -0.23920460544530125, -0.05700162248003914, -2.041801518842504, -0.4250028163147696, -0.3106047989802361, -0.3099312826625347, 0.6384891532705531, -0.45900067903892555, -0.7015706640524254, -0.4396629750031297, -2.311708316689941, -1.066804192450254, -0.054569061194009755, -0.28648676012083424, 0.09670707981634862, 3.408643407834532, 0.31640831048864804, -0.28271576166671225, 1.63461362334252, -0.5752250493637142, -6.9732938706004735, -0.008352436004511252, 0.004407119942774197, 0.6205331233624234, -0.6074985366253337, -0.18753092625810105, 0.19392224286204843, 0.1685749461015153, -1.3601339503888812, 0.36388947643287034, -0.06196871107305454, -0.04771327522057511, 3.0617441447545297, -0.2900360370157078, 0.8227975928308909, -1.1108577013255012, 0.4088013642781272, -0.2540477734777634, -0.5115280779864364, 0.3611938318336944, -0.44057457557103036, -0.9520013184607715, 2.5022814679732566, -1.2806511054964937, -0.005719097996433783, -0.3197043353644746, 1.358134547692764, 0.8261110453192144, 0.14308159459960257, 0.1888386743281272, -2.287746011205473, -0.6153281683125567, 0.4935108153755977, -1.534732865864548, -0.3418009463087599, -0.7336794071875923, 0.7028985019256049, -1.2235158802135626, -0.8025699517095539, -0.35436664835734694, 0.9451138186303847, 1.3429615213268167, 0.8318108654057249, -0.12659062644712682, -0.48584763349031657, -0.8704388740711496, 0.20947755072518084, 0.5392662190278082, 1.3612736205192206, -0.18253175451565828, 1.5506770400166636, -0.07686396575411233, -0.6958492891965404, 1.9054527836489343, -0.15826965947497748, 0.7274322480252438, -0.01027972070375067, 1.8280286092639877, -0.31465527448662894, -0.2708812017482421, -1.5411788819161725, -1.079077831026048, 0.3396481721433586, -0.3513162213341439, 0.6413305992620802, 0.1918346546781671, 0.25573877606169115, 0.2304470900832106, 0.5030963843115569, -1.1274802764953957, -0.3707299695513268, 0.11434351515775043, -1.4547184978360759, -0.4358695425230781, -0.8903953567486279, -0.4290819671840667, 0.28382385690487316, 0.12706047702123938, -0.8484472014884187, 0.24221912846233412, 0.0937703800988815, 0.2335944725827487, -0.08419980209394823, 0.7972443860047689, 0.21148503785811293, 0.2681804563530993, -0.18709355357390736, 0.2805416765541082, 0.2361293381026413, -0.7020251428249609, 0.04350858391761822, -4.219165549368773, -0.19779585180007941, 1.8142745958311237, 1.2197369770959519, -3.719342620425578, -2.8353807448982558, -0.17958261724750244, 0.3331827610400401, 0.8040699845063418, -1.0528872985772453, -1.0660153356227429, -0.6375713995706666, -0.26333599414399056, 1.2918674352262265, 0.0572670223670812, 0.02550007217970974, -1.4321607739195192, 0.5771954331289317, -0.03421568181493331, -0.3309463993329558, -0.17953563758536975, -0.4522798454577029, -0.2107817282897452, -1.3151365775882302, 1.1655494668428217, -0.02180570066817121, -1.4898573604007639, 0.13181403164239866, 0.9333727563347053, 0.3882205103254239, 1.92430059831509, -0.3112783365463154, 0.9132176604013629, -0.2986830230515119, -0.28398674573530275, 0.7776927906049077, 2.2055788671346845, 3.2407448399217187, 0.25479262324738605, 0.6005919809145499, -0.8869172441185057, -0.07042572126538016, 0.4075481722540547, 0.19205652724410938, 0.21327355965318273, 0.711556788206964, -0.6378957742028838, -0.41981260988659397, 0.7568564000750936, -0.6104798852681452, 0.9847068188361395, -0.4585265086300011, 0.01751829303842508, -0.967448731937149, 1.5560905251361346, -0.8399278380084444, 0.8271995395677773, 0.15932083709698616, 1.1948682965222683, -0.004069520400809879, -1.2906790492198819, 0.1876206649824473, -0.2986419372626623, 1.1966700680426674, 0.5288966350480795, -0.36247254833799086, 1.8122086716829482, -0.7114693317676599, -0.6687422486471285, 0.5987076512031672, 0.3657739423018651, 0.18863180733981966, 0.17817410486820934, -0.4998685868619051, -0.15784158892010441, 1.4422938932641212, -0.584618989064135, 1.308360558246265, -2.300444382052494, 3.3376371555937694, -0.6037003078070785, 0.6171503932430207, 0.30680148187080225, -0.9168196752578966, 0.9156001797908863, -0.0078435219289687, -0.5394507171017716, -6.2686411606514945, 0.6342703463585777, 0.46363211902341883, 0.2565747826310927, 0.057283316473497124, 1.6885773260170955, 1.5271116502354267, -0.6370235945260564, -0.3358624152572086, -4.234871662323356, -1.1083888610667718, 0.5782736483923052, 0.1467741841818383, 1.9758109820125713, -1.2367194817009028, -0.8003485753811507, 0.15343869845611804, 0.5975439284684059, -0.5221391084453075, 1.447683514307985, 0.7897088104237512, 0.2716650325295809, -0.09966678458224137, 1.3292463257520977, 0.08200206401081465, -0.30673472306465727, 0.4260301325806802, -0.1661665282705384, -0.37745198399663005, 1.075580222245388, 0.71601920166168, -0.3198475403321577, -0.22703988719197574, -0.19205808406801214, -0.17521186400116823, -0.9889655085869868, -2.933649205809281, -0.07667840485583982, 0.5197680538582372, -0.24070755995622334, 0.011082824527219889, 0.27098922308730355, 0.6936356475424426, 1.5332062462502305, 0.5918322815861793, 2.9206206036971327, -0.3008903502002213, 0.6870373872935148, -1.0580138264164594, 0.9661213402435523, -0.22639216437109377, -0.9606532684073648, 0.3523335596379108, -0.37631254360940325, 0.757649514116488, 0.6368437079092006, 5.554588803765649, -1.987288750581721, -1.0096896551828742, 1.0169022264403085, -0.16955238073401083, 0.04610023665622286, 0.612877587622959, -3.9177047058155283, 1.569246793975406, 0.12123111581470476, 0.16508572847144953, -1.1265417678381213, -2.6583768621700563, 0.3256559445232512, 2.378681087182308, -1.4658834622397054, -0.35995300939972147, -3.1753744888470536, 0.7814803826088135, -0.2531999321156395, 0.16982396685709478, -0.3989150357771902, -0.28246419219156776, -1.581598353104042, 0.16255920110944475, 0.12964268923783842, -0.8579317318470369, 2.9316556899276813, -0.09292606983324851, -0.5818655465755689, -3.600095934593008, 0.4263875868430066, -1.1890507570359776, -0.03212663639924068, -0.5251917696732407, 0.3471343677017093, 1.536416581981694, -0.23208031309745317, 0.6403371397499374, 0.028256672129783417, -1.6585461228441931, -0.13340668757038046, -0.5660289129007874, -0.061650713498533224, -0.6448218149640644, 0.637670332751796, -0.7726352375157887, 0.571760708564402, 2.0275769057186857, 0.48880609708795436, 2.540035090734576, 1.5885244115088146, -0.48189594042627887, -0.09312820149706567, 2.0933511400839437, -0.7733920204565466, 1.2338108905032477, -1.0737836075723888, -1.6615652803540109, 0.4029247008450761, 2.1928513524155218, 0.628440156578093, 0.7694698640476578, 0.07918112199063858, 1.1213411015450856, 0.03221220164944722, -0.2473349024779808, -0.2649746342967478, -0.19352522860966098, -0.07891263332317287, -0.5367203817368585, -0.09982376479670695, -0.03928371992595013, -1.3933587104607552, 1.5031712873584842, 0.2524329632242526, 0.3362759780662619, -0.034459256653807446, 0.2371837818970316, -2.17150197752564, -0.17703545007093552, 0.4240441718732759, 0.5498280712965592, 0.09025376892074514, -0.5273917595263548, 0.9205353285180912, 7.5161208011782215, 0.3687697521969073, -0.60749238518603, -0.5770141935847835, -0.07350372618196124, -0.2580080011370105, 0.6902919492087574, -0.35681620806047876, -0.04428542650178506, -0.17792060109978738, -0.9145310342171994, 0.5856601503555662, 0.7842263303645082, -0.3663271154349017, -6.73729590148993, 0.26440702262887567, 1.5722623952747428, 0.4065396647852953, -0.6129862728415989, 0.1321049352084805, 0.6044547092212715, -1.2374241963058048, 1.2226529669930382, 2.604499807624791, -0.525645251213192, -0.04328324757151574, 0.44249045715594026, 0.42652775932942166, -0.2513624831510135, 0.7360009836745844, 0.8172020805677506, -0.6267248303348356, -0.7880591434416133, -1.4878989465783352, 0.20649931503253785, 4.275655885319044, -0.36438359458603753, -0.2009336087347416, 0.5042564148037928, 0.41284005796057904, -1.09481560485961, -0.6017381977210495, 0.26468053686606474, 0.4056189492307689, -0.23960771048315593, 0.12271074656207397, -0.15538345588681726, -4.144848302860195, -0.6214501049986082, -0.9647274361055707, 0.4355563631668005, -1.2855983785146032, -0.8903870863406019, 0.09742518197680325, -0.5682903923785899, 2.1416602700600094, 0.07419646841545163, -1.4637350473830373, 0.3166817347863247, 0.07697742538487103, -0.338844476007037, -0.16407341068913764, -1.1449237440109117, -1.4933376796748379, 0.027648613460829686, -0.9346803826485007, -0.06617501481662937, -1.1312637837150834, -0.06658354231924177, -0.19853317102890697, 1.0331225965194588, 0.9216481777427576, -0.06239023023323874, -0.5461138119232651, -0.5613627022502932, -0.38253669287131886, -0.49449354733905176, -1.1991761333596513, -0.4986469217607072, 0.6656591460297222, -0.3102603713997906, -1.6323814422022915, -0.13337608398500525, 0.9516555078891815, -0.4104611126811158, 0.49752467309565857, 0.027170510168600864, -0.977836827721432, 0.3432309029464574, 0.9120595811906369, -2.213532272747819, 0.48501564205868586, 0.2328490371217075, 0.14637802600776811, -0.6341732616003616, -0.4418891847409728, 0.08169670905596282, 0.24854188590240262, 1.5561648165294972, 0.0765661088072844, -1.1246504365141399, 0.6531275355009007, 0.31350870375234857, -0.35059681016393857, 1.6052771958025165, -2.3369030946546805, -0.47724923206821557, -0.9783604684072394, -0.4396634954884326, 0.36688127842994284, 0.8706733047494032, -2.7518247324015617, 0.37416749651161363, -0.7429770053626444, 0.1474582265390092, 0.4431867443529597, -0.09956539095820408, 1.1244514122353053, 0.02762830237593174, 0.8069574335381544, 0.3641015475871204, -0.7406840006769995, 0.028362328844444867, -1.0700487178022497, 0.18668333807761708, -2.932727884621628, -0.38592317297323153, 0.16678026187993172, -0.13800476334412407, -0.3724404368910663, -0.23554437465367828, -0.10264152565769155, 0.033719050535941514, 0.40215667883143763, 0.7231280188693898, 0.45754586035323935, -0.5281735444479623, 0.19305905209915933, -0.5546041559660015, 1.6812264962225603, -0.20613145168382066, -0.8488942415866223, -0.966386109939005, -0.5647219079191913, -0.21199455517020935, -0.24376218227305108, 2.2962965138208764, 0.315721700787881, 0.21422977993036602, 0.008845693881800462, 1.29913346808983, 0.41338269302041425, 0.37990071615518894, 0.08315164837452879, 0.23849806187297945, -0.6072374046036046, 0.22188932634757608, 0.16416421377007734, -0.6247432077041496, -0.5873382602674674, -0.18985586791291617, -0.0055279103873100335, 0.3900975954924062, -0.3897220788480643, 3.2688498757328346, 1.5627518640044447, 0.4500664805550119, -0.4565464537057052, -0.8638523547393819, 0.3997515610860481, -2.3366381805193632, 0.1911844822648328, -0.5432155974374557, -1.0032228586164165, 0.5443706853850342, 0.5347187222017283, 1.219121525433102, -1.26721826937007, 0.2509419294350671, -0.8057700254060013, 2.8554922071967153, 0.5038773875945092, 1.291194180291576, -2.1500449258799144, 0.6858297752980795, 0.9166921191933595, 0.3317957998992648, 0.33742153788911083, 0.5471107958191145, 0.27317819343600896, -7.20924340050665, -0.8582256906036714, 0.35941413691937735, 0.09477140842060369, -1.420976104195515, -1.2345344897304291, 0.36372809410380813, -0.2135769930758346, -0.9675320335725127, -0.2370397545098121, 0.6515916584334261, 1.5450570452923873, -1.5314707886718903, -0.4450114677769828, 0.6831076272180514, -0.5438100883089853, -0.025581717108371448, 0.16213125912017448, -0.6029233650219649, -0.30444758838103037, -0.15598477762218313, 0.8791416721187839, -0.007281183518743411, -0.6681774633894935, -0.6308895467449046, -0.21054913026784025, 0.1603803164201271, 1.2964772042528048, -0.5322245970326815, -0.16640383726239696, -0.6354572312739154, -0.5519901190521328, -0.0800701010738623, 1.874071778780899, 0.4971013583477369, -1.6386421714585706, 0.5959380797019479, 0.45299833214015506, -0.42132198254167413, 0.09476390832017517, 0.4464357808412932, -0.23432363509373, 0.4557364666209492, -0.08069420596232006, -0.5408838554612727, -0.21903357963482387, 0.5970481597521154, -0.18985184300424587, 0.5057254011936663, -0.8349313471200048, -0.90777958492981, -1.4539994200082131, 0.28568623214170835, 0.11389153471216587, -2.6260512746727738, 1.2704238656573703, 1.2805344482081815, -0.27201384042895294, -0.22580131494148298, 0.4285113451079058, -0.8161983264852594, 0.6721709566934618, 3.795011299373442, -0.5582952457376439, 0.11518855372861256, -1.5589978895776326, 0.018236508171771717, 2.7529884333763124, 0.7225690978303176, 0.34284945537050815, 0.7161684294459116, 0.8715307237126144, -1.1248137677452803, -0.03740160675177632, 0.5146013519332093, -1.2078434862401135, -0.43287499351887526, -0.0807176324935513, 0.1566580729160793, 1.27471693531688, 4.296261318633602, 0.8894205217848474, -0.5283939681579942, 0.35917607265399476, 1.3750623424817177, 0.8230357550937814, -0.16826245980026303, 0.9750691686309036, 0.9407391959889159, -3.5094644590263404, -0.19634474268234958, 1.7645230283588544, -0.1203412626944502, 0.08059151691589116, -1.3964991204566386, 0.8026973624597655, -1.0735075166753776, -1.1372451015831204, -0.09422116487876991, 0.3789096005209706, -0.3173694605263983, -1.2481338658228796, -0.3518328870124035, -2.1150312638520745, 0.26708866167978856, 2.737357249736439, 0.6467386951151068, -0.12048218926079872, 1.4351168840795359, -0.6476597697267686, -0.048961332897683424, -0.5534423030113818, 0.37992703803137057, -0.47766467266318907, -1.0063842234458742, -1.3293518853315571, -0.1217680112293993, 2.351112750955019, -0.04159272452022674, -0.7649093899192598, 0.7415190758624463, -0.9427954702232012, 0.0485231049327464, -0.1570290558379509, 4.098734193774656, 0.893024171166602, -0.40452156062000355, -0.29508792459659583, -0.3513376806555928, -0.11709017931493064, -0.36330394989894294, -0.07814363386384623, -1.0838580466133847, 0.5799195417823805, -0.4415609794542585, 0.40890903793786665, 1.8956436601098043, 1.3102263282356397, -1.6383416175428922, -4.8211016898348165, 2.370434932833576, -0.2869774937184653, -0.5522575920115121, -0.23770032844542008, -0.7916653653443363, 0.697021707628647, 0.37929150741798806, -1.7888400370493398, 0.5386703525284665, -0.5015415041119434, 0.056646428346639834, -1.0394434860408879, -0.40234381462559893, -0.11316407906007354, -0.1798674471744491, 2.542276126351221, 9.999104495940275, 0.5998129031933956, 0.3739515269800114, -0.10990728269667902, -0.3424989451212019, 0.19178042716603186, -0.13525784129940735, 1.3745276553127634, 1.9251327259061435, 0.5485990605874909, 0.4638870132312199, 0.340897432779037, -0.30783759094085617, 1.8212037425981273, 3.263703095197062, 1.2024762696032008, 0.23824509241499758, 0.727376134856329, -0.8361945766113977, -0.2948444762671517, 0.7741822400170436, 1.3883458946700857, 1.8507472361662698, -0.1545414256534021, -0.013628507713726895, 0.6018881284258748, 0.33717187201150595, 4.86288663238002, -0.7114022414052924, 0.5003959700126239, -0.585599552016286, 0.08141946265163615, 1.3340644736677392, -0.6373140675458104, 0.16341700911360685, 0.2902361414376804, 0.6652420226575737, -0.38922642966139875, 0.3371530821971621, 0.7727708376248499, -0.026299212447486543, 0.28193243379817357, -0.22152377110864732, 3.1430506237444567, 0.9131853819280936, -0.40905151573257953, 0.03702629862105689, 1.3166353939357265, 1.3480072699378973, 1.6839598532406796, 0.9319194499733828, -0.2493060089056907, 0.932004630918625, -0.4952648257666049, 0.12825573810062157, -0.4158904740508131, -1.3867542454163049, 0.6835198522645953, -0.6832698102450915, 0.3343079176217418, 0.8063406486720064, 0.9930833779628543, 0.1604151529823607, -0.33797811864550564, 0.3847079917455164, 0.16661244512067833, 0.6628029790867694, 0.2669169789413303, 0.07769050021544072, 0.4743664588859619, -0.4384617126449608, 0.2263316797915816, -0.3678051703071583, -0.41838770227338173, 0.19593869633307345, -0.0078113727066504495, -1.6642007765628826, 0.6366861758921255, 0.2979324806937917, -1.1866810300780348, 0.7640704407722604, -0.12543922811867245, -0.578486411767342, -0.3116661929628721, -0.19817017569834702, 0.26759065165399715, 0.20670757941920717, 5.771220367031614, 0.24836731609530152, -0.06947901966253342, 4.5474342984891445, -0.8017976155530493, -1.5307164515868987, -0.22486588553557113, 0.037653988574503965, -0.65982725518912, 0.5022381138341081, -0.1188328448157119, 0.212084857159307, 1.7688960483048988, -0.3162145762124658, -0.42697365386969977, 0.03085539242265478, 0.3187498037377755, -0.5056660446578116, 0.805973379930502, -1.5736458159278428, 0.7068922623560943, -1.0260848896234624, -0.2314592980352821, -1.3970645781777127, 1.1766247656277762, 1.2477979413178137, 0.42929538691851155, -1.6410219708814968, -0.9387171434700478, 1.9395015983214268, -0.3730567688950866, -0.08206341500097351, 0.7231707462551913, -1.2139164292590856, -0.5876535217141263, -0.36150707413276495, -0.24239041513932635, -2.093325781287194, -0.35958298680035117, -0.6361993914669322, -0.6978654835114272, -1.3866332011205031, -1.7727543428919925, 0.09322059175015283, 0.5680169883543331, -0.7273939113975875, 0.4151478572925792, 2.4378959872340267, -1.1841070181066387, -0.21445355084719214, -0.2944836707352156, -0.6706288413950168, 0.22197723104308467, 0.014034653717267928, 0.593553583753287, -0.06329758030306883, -0.4967254177911602, -0.5414406235312106, -0.1943496253949531, -0.4510266077485711, 1.7759471342961257, 0.02029559871072448, 0.12344355655339159, -3.9644655072659845, -0.1550099490644497, 0.1839996250342466, 1.1698319131588242, 0.3782056920889422, -1.5508711217315188, 0.6962937923548856, -1.0819356270972742, -0.0018272408232390981, -2.1124753052277443, 0.9132022659676244, -2.1789199296875865, -0.31790046887299184, 1.9699842167824346, 3.143663198086899, -0.996458461734132, -0.12263182829697628, -1.0779450378906261, 0.47520193253345083, 0.1265873493740738, 0.05025035307321422, 1.8696347771310107, -0.3134827985515733, 2.02427809884913, -0.09799528229259047, 1.199408459365537, 0.3914376543779331, -0.5713156615380921, 0.5549795325267844, 0.932847590433125, -0.7066687311889591, 0.6944372647569452, -0.03144663150050604, 1.609363892127189, 2.1371453721779625, 0.11722879810771922, 0.5855114304195245, 1.3437323146566005, 0.01733057839066746, 4.1350061040004285, -0.08295876135219904, 0.41705137103117174, 1.0069964763163342, 0.8584687172213588, -0.5455480053545245, -0.09502212716772633, 0.02543336041769497, 0.9976093953456064, -0.8171730795100098, -0.13133501047751514, 0.7848714572195755, -1.4390776966848053, -9.998765907811515, -0.3020886729273453, 0.6304738715290689, 0.3683307106787502, 0.056210131275333045, -0.7271383577015476, 0.4629881951508561, -0.5967107212711619, -2.2506673427395576, 1.0198543550070895, -0.46160792680239904, 0.26450908621284813, 0.8046112012900201, -0.08028860861740283, 0.3424981515558451, -0.9265234098883738, -0.6462988239089081, 0.41127140469771445, -0.024892967594502068, -0.45480703275071616, 0.9367221449019854, -2.207313220697821, -0.9981654402858081, 0.07975414597446252, 0.5939954671959319, 1.0736335494340798, -0.4721166310767902, -0.2311599692752705, 0.5549728389078306, -0.50651272834137, 1.0156036509425852, -0.12062550818449792, 0.6844217016141534, -0.30019348896096226, -0.11205461585777535, -0.005454689419279807, -0.24940859271150898, 0.6955194969521651, 0.12808916364279083, -1.0649567842700955, -0.20219108454773752, 0.5876030223423053, -0.025838864752254956, 0.01633081623663012, -0.9160647687068844, -0.897266324647235, 0.1733137694236489, 1.4253281394819763, -0.30095844705211816, -1.0787582848277706, 1.2885956813968407, -0.5586567450314752, -3.4931809885538128, -0.3496811944765368, -0.024480537355198426, 0.08201706905278293, -2.3924447497232944, 1.043501156512884, -0.015779284523925463, -0.606632100365044, -5.240570209513608, -0.15557450964346656, -0.7952471205360658, 0.1080136065532577, -0.42564864908704614, 1.9176238447646836, -0.4656443271187999, 0.2014177679152012, -0.194571550864344, -0.11412792164334305, -0.23135496956203858, 0.15228657701425874, 1.082706654523902, -0.6295532541161589, -1.1515973718645964, 0.015065606086850251, 0.789322109070939, -1.0706922279674262, 2.0216903939535102, -3.0766301360804973, -0.4531072279094747, 1.0455009578771108, -0.7473320145897943, 1.709706471250618, 4.837143000034951, -0.27616117905171994, 6.358789275015443, 1.3590618933605887, -0.12197497034556683, -1.1090385740556645, -0.385698970598323, 0.03619392583794699, -0.31600302891487947, -0.47659902393259385, 0.4451615391017806, 0.7493111892282751, 0.33820761574036406, -1.0516988906549214, 0.11482961001568125, -0.4558707552128471, 0.06504750856807587, -1.4406018988660128, 0.440506754378568, -2.9571273534789686, 4.892832157818674, 0.9157339568348827, 0.2874542441512834, 0.14689139506453897, 0.11023559897018019, -0.44389452230243653, 1.5552253685229884, 0.10480395355206465, -0.23700121707743232, -0.4658533639888576, -0.028707465174578065, -2.592713587912742, -0.9698249919581832, -0.2528002638275678, 1.5000001623059773, -0.20767611159298416, -3.113334619584772, -0.2939836063808369, 0.09111053824530259, -0.8452347283405999, 0.12200704182398742, -1.1272999407663655, -1.51846203509101, -0.6929484518170822, 0.6744291936169868, 0.554700896992181, 1.7036033971155269, -0.38115087431692923, 0.6207297660970703, -0.2011549763884225, 0.9889744198940554, 0.14631011787218423, -0.6923953846408769, 4.2582444460117115, 1.4460509312923686, 0.25093574401607194, -1.2187000843930313, -0.6171827526378366, -2.754406881893992, 0.011578844205319669, 2.58696982144323, 0.4551046021894294, 0.5824938638826054, -0.8863603668677038, 1.4904162857234493, -1.1056956072152122, 2.859730684007067, 0.6585891343602494, -2.800329078592394, 0.5607063634440891, -0.4252981196646224, -0.5184262096942905, 1.0986637868387914, -0.21271891817241856, 0.4707031114006687, -0.208710431787659, -2.6388841009154813, 0.42331485673582975, -0.3452715956293512, 2.8790558893343574, 0.3012633277301409, -0.2869658412800642, 0.17536064042905247, 0.07602409622037137, -0.024575701185477576, 0.07972278240743384, -1.0048256729472564, -4.097096572894135, 0.2712478556415644, 0.39865612948204937, 0.6532492171625632, 0.22478004004798363, 1.4072761494641295, -0.05587438972966361, 0.1915244335150561, 0.053838480573740556, 1.3745425022308462, 0.8776326773310198, 0.5840981169142853, -0.22222219507559215, 2.018081889000074, -1.8936489059883321, -8.768153383460394, -1.6203859810672552, 1.664440438829625, 0.039691337040904676, 0.48731856671361357, -0.23943921071965243, -0.7961376470972941, 0.0022809685070282527, 0.027277126261753454, 0.8338337140089478, -1.1185452560493359, 0.8071945044680331, -0.28224520491476945, 0.3905403099132002]
#load the Nepal alpha matrix which already has the common alpha_ij inserted from Brazil, Denmark, Mozambique and Iran alphas
Nepal_alpha = np.loadtxt(open("fix_alpha_Nepal.txt","rb"), delimiter = ",", skiprows = 0)

#insert 2860 alpha_ij predicted in this step
for i in range (0, length):
    coordinate = position[i]
    Nepal_alpha[coordinate[0], coordinate[1]] = opti_Nepal[i]

#save Nepal alpha matrix
np.savetxt('Nepal_alpha.txt', Nepal_alpha, delimiter = ',')

In [8]:
#test
#import the matrix alpha
Nepal_alpha = np.loadtxt(open("Nepal_alpha.txt","rb"), delimiter = ",", skiprows = 0)

#compute the Lambda invasion matrix
lambd = np.zeros((strain_number, strain_number))

for i in range (0, strain_number):
    for j in range (0, strain_number):
        lambd[i][j] = mu*(Nepal_alpha[j][i] - Nepal_alpha[i][j]) + Nepal_alpha[j][i] - Nepal_alpha[j][j]

In [9]:
# test
# compute the equilibrium of predicted model
one = np.ones(strain_number) #vector of all ones
inv = np.linalg.inv(lambd) #inverse matrix of fitness matrix
z = (1/np.matmul(one.T, np.matmul(inv, one)))*np.matmul(inv, one)

#compute error between observation data and equilibrium of predicted model
error = Nepal_cases - z
error = np.linalg.norm(error)
print(error)

0.010086222466852413


In [16]:
#compute Jacobian matrix corresponding to the equilibrium
jac = jacobian_func(lambd, z)

#compute eigenvalues of Jacobian matrix
eigenvalues, eigenvectors = np.linalg.eig(jac)
real_eig = eigenvalues.real
eig = real_eig.tolist()

#find the max value of eigenvalues' real parts and print it
max_eig = max(eig)
print(max_eig)

-0.0003445557932257083


In [17]:
#test if there is any real parts non-negative
positive_eig = [i for i in eig if i >= 0]
print(positive_eig, len(positive_eig))

[] 0
